In [12]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import os
from sklearn.linear_model import LinearRegression

from scipy import stats
from scipy.stats import norm, ttest_ind, shapiro

In [17]:
path = "C:\\Users\\ROfl\\Desktop\\MS\\MatStat\\4"
os.chdir(path)

CFD = pd.read_excel("DATA.xlsx", sheet_name='ДАННЫЕ')
CFD.columns = ["STAF","EDU","GEN","SAL"]
n = 509
CFD = CFD.iloc[: n]
CFD

,STAF,EDU,GEN,SAL
0,22.384615,12,0,53.410000
1,8.903846,12,0,8.000000
2,13.250000,15,0,24.000000
3,18.250000,13,0,29.500000
4,13.769231,18,0,32.049999
...,...,...,...,...
504,16.019230,16,1,23.250000
505,17.557692,17,1,48.070000
506,18.942308,12,1,11.500000
507,17.442308,12,1,9.250000


In [25]:
alpha = 0.02
gamma = 0.975
n = 509

In [13]:
model = LinearRegression()

In [27]:
X = CFD[["GEN"]].values
Y = CFD["SAL"].values
model.fit(X, Y)

b_0_hat = model.intercept_
b_1_hat = model.coef_[0]
print(f'Оценка свободного члена (b_0): {b_0_hat:.4f}')
print(f'Оценка коэффициента при Z (b_1): {b_1_hat:.4f}')

# Коэффициент детерминации R^2
R2 = model.score(X, Y)
print(f'Коэффициент детерминации R^2: {R2:.4f}')

# Проверка гипотезы о значимости b_1
n = len(Y)  # Объем выборки
y_pred = model.predict(X)  # Предсказанные значения
errors = Y - y_pred  # Остатки
# Стандартная ошибка коэффициента b_1
S_e2 = np.sum(errors**2) / (n - 2)  # Оценка дисперсии ошибок
X_mean = np.mean(X)
S_b1 = np.sqrt(S_e2 / np.sum((X - X_mean)**2))

# t-статистика для b_1
t_stat = b_1_hat / S_b1
t_crit = stats.t.ppf(1 - alpha / 2, df=n - 2)

print(f't-статистика: {t_stat:.4f}')
print(f'Критическое значение t при alpha = {alpha}: {t_crit:.4f}')

if abs(t_stat) > t_crit:
    print("Коэффициент b_1 статистически значим.")
else:
    print("Коэффициент b_1 незначим.")

Оценка свободного члена (b_0): 23.1145
Оценка коэффициента при Z (b_1): -6.9108
Коэффициент детерминации R^2: 0.0551
t-статистика: -5.4348
Критическое значение t при alpha = 0.02: 2.3337
Коэффициент b_1 статистически значим.


In [30]:
# Параметры модели
k = 1  # Количество объясняющих переменных
# F-статистика
F_obs = (R2 / k) / ((1 - R2) / (n - k - 1))

# Критическое значение F
alpha = 0.02
F_crit = stats.f.ppf(1 - alpha, dfn=k, dfd=n - k - 1)

In [31]:
print(f'Наблюдаемое значение F: {F_obs:.4f}')
print(f'Критическое значение F при alpha = {alpha}: {F_crit:.4f}')

if F_obs > F_crit:
    print("Модель обладает высокой объясняющей способностью.")
else:
    print("Модель низкого качества.")

Наблюдаемое значение F: 29.5373
Критическое значение F при alpha = 0.02: 5.4463
Модель обладает высокой объясняющей способностью.


In [37]:
Y = CFD["SAL"]
X = CFD[["STAF","EDU"]]

In [38]:
# Модель линейной регрессии
model = LinearRegression()
model.fit(X, Y)

# Коэффициенты модели
print(f"Свободный член (beta_0): {model.intercept_}")
print(f"Коэффициенты (beta_1, beta_2): {model.coef_}")

Свободный член (beta_0): -27.455478527947783
Коэффициенты (beta_1, beta_2): [0.59154003 2.72649098]


In [44]:
print(f"Изменение зарплаты, если стаж увеличиться на 1 год, при неизменном образовании: {model.coef_[0]}")

Изменение зарплаты, если стаж увеличиться на 1 год, при неизменном образовании: 0.591540026520532


In [45]:
print(f"Прибавка к зарплате при доп годе обучения, при неизм стаже: {model.coef_[1]}")

Прибавка к зарплате при доп годе обучения, при неизм стаже: 2.726490975272277


In [51]:
k = 2
F = (R2 / k) / ((1 - R2) / (n - k - 1))
print('F-статистика:', F)

df1 = k
df2 = n - k - 1
F_critical = stats.f.ppf(1 - alpha, df1, df2)
print('Критическое значение F:', F_critical)


#H0: Коэффициенты при факторах равны нулю, модель не объясняет зависимую переменную
#H1: Хотя бы один из коэффициентов не равен нулю — модель обладает объясняющей способностью.
# Проверка гипотезы
if F > F_critical:
    print("Модель обладает высокой объясняющей способностью (отвергаем H0).")
else:
    print("Модель низкого качества (не отвергаем H0).")

F-статистика: 14.739536583495035
Критическое значение F: 3.942424406703579
Модель обладает высокой объясняющей способностью (отвергаем H0).


In [53]:
beta_hat=model.coef_

# Оценка остатков и дисперсии
n, k = X.shape[0], X.shape[1] - 1
y_pred = model.predict(X)
residuals = Y - y_pred
sigma2 = np.sum(residuals**2) / (n - k - 1)

# Ковариационная матрица коэффициентов
XTX_inv = np.linalg.inv(X.T @ X)
cov_beta = sigma2 * XTX_inv
se_beta = np.sqrt(np.diag(cov_beta))

# t-критическое значение
alpha = 1 - gamma
t_critical = stats.t.ppf(1 - alpha / 2, df=n - k - 1)

# Доверительные интервалы
conf_intervals = []
for i, beta in enumerate(beta_hat):
    margin_of_error = t_critical * se_beta[i]
    conf_intervals.append((beta - margin_of_error, beta + margin_of_error))

# Вывод результатов
print("Коэффициенты регрессии и их доверительные интервалы:")
for i, (beta, ci) in enumerate(zip(beta_hat, conf_intervals)):
    print(f"β{i}: {beta:.4f}, ДИ: ({ci[0]:.4f}, {ci[1]:.4f})")

Коэффициенты регрессии и их доверительные интервалы:
β0: 0.5915, ДИ: (0.3676, 0.8155)
β1: 2.7265, ДИ: (2.4445, 3.0085)
